In [73]:

import os
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append('../..')
confluence_username = os.getenv("CONFLUENCE_USERNAME")
confluence_password = os.getenv("CONFLUENCE_PASSWORD")

from llama_index.core import download_loader
ConfluenceReader = download_loader('ConfluenceReader')

import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract' 

/tmp/ipykernel_4741/2857989293.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  ConfluenceReader = download_loader('ConfluenceReader')


In [55]:



base_url = "https://wiki.softtech.com.tr/"
space_keys = ["YPZ"]#, "YNYGAB", "SPISS"],["YPZ"]
reader = ConfluenceReader(base_url=base_url,  user_name=confluence_username, password=confluence_password)
spaces = {}
for space in space_keys:
    spaces[space] = reader.load_data(include_attachments=False, space_key = space)

for docs in spaces.values():
    for doc in docs:
        print(doc.extra_info)


{'title': 'AI Lab', 'page_id': '86148988', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//display/YPZ/AI+Lab'}
{'title': 'frAIday', 'page_id': '89752943', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//display/YPZ/frAIday'}
{'title': '2023-05-12 Haftalık Statü ve Paylaşım', 'page_id': '89753887', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//pages/viewpage.action?pageId=89753887'}
{'title': 'Toplantı Kayıtları', 'page_id': '89754463', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//pages/viewpage.action?pageId=89754463'}
{'title': 'Google AI süreçlerinin deneyimlenmesi-1', 'page_id': '89754465', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//pages/viewpage.action?pageId=89754465'}
{'title': '2023-05-26 Haftalık Statü ve Paylaşım', 'page_id': '89754714', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//pages/viewpage.action?pageId=89754714'}
{'title': '2023-06-02 Haftalık Statü ve Paylaşım', 'page_id': '89755436', 'st

In [31]:
for docs in spaces.values():
    for doc in docs:
        print(doc)

Doc ID: 43322693
Text: # **Unity in Diversity**  Architecture is about making
intelligent trade-offs to manage change within a system, and it is
important to be pragmatic about where you invest your time and energy
when you are delivering business value.  Do not start with
microservices! Fewer moving parts (smaller control plane) are easier
to debug and understand.  I...
Doc ID: 43322699
Text: **Event Store Usages**          # List of Tactics  Behavioral
Tactics          https://docs.microsoft.com/en-
us/azure/architecture/patterns/
Doc ID: 43322708
Text: # Intent  Explicitly differentiate between symptom ( _effect_ )
and cause:    * Identify root causes of symptoms, problems or issues
* Trace a problem to its origins  # Description  Some people fight
problems, not their real cause: When faced with a problem our brains
tend to start immediately to search for proper solutions for exactly
this p...
Doc ID: 43322722
Text: # List of Tactics  [en]Root Cause Analysis| Explicitly
differentiat

In [62]:
spaces = {}
for space in space_keys:
    spaces[space] = reader.load_data(include_attachments=False, space_key = space)

In [63]:
from pprint import pprint
for space_key,space_docs in spaces.items():
    print(space_key,len(space_docs))

    pprint(space_docs[0])


YPZ 75
Document(id_='86148988', embedding=None, metadata={'title': 'AI Lab', 'page_id': '86148988', 'status': 'current', 'url': 'https://wiki.softtech.com.tr//display/YPZ/AI+Lab'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="AILAB Kimdir?\n\n  * Yapay zeka vizyonunu ve hedefininin iletişimini yapar\n  * Yapay zekanın etkinleştirebileceği yeni teknolojileri, kullanım örneklerini takip ederek yenilikler yapar\n  * Mail grubu : AILAB Geliştirme Ekibi <ailab@softtech.com.tr>\n\nMiro\n\nhttps://miro.com/app/board/uXjVMowFuqc=/?share_link_id=910654917308\n\nBoard\n\nAI Lab - Agile Board - Jira (softtech.com.tr)\n\nDiscord\n\nhttps://discord.gg/5njRbhxbpP\n\n  \n\n  \n\n  \n\n* * *\n\n### AILab ile nasıl çalışırım?\n\n  * FrAIDay'lere katılırım ve gündemi yakın takip ederim\n  * Benim için tavsiye edilen eğitim ve videoları izlerim\n  * Duyurulardan haberdar olmak için **AI Çalışma ve Bilgilendirme <aibilgi@softtech.com.tr> **grubuna kendimi ekletir

In [64]:

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_milvus import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2")
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

abc = 0
for space_key,documents in spaces.items():
    chunks = []
    for document in documents:
        title = document.extra_info['title']
        content = document.text
        source = 'confluence'
        doc_id = document.doc_id
        url = document.extra_info['url']
        # [docs.extend(Document(page_content=chunks[i], metadata = {'source':file.name, 'page': i}) for i in range(len(chunks)))]
        metadata = {'title': title, 
                'source': source, 
                'doc_id': doc_id, 
                'url': url,
                'Header_1': '', 
                'Header_2': ''
                }
        
        md_header_splits = markdown_splitter.split_text(content)
        if abc == 0:
            for md in md_header_splits:
                print(md)
            abc = 1
        for i, split in enumerate(md_header_splits):
            # print("sub_id den önce", metadata)
            metadata['sub_id'] = i
            metadata.update(split.metadata)
            # print("sub_id den sonra", metadata)
            if not (metadata['Header_1'] or metadata['Header_2']):
                page_content = f"Introduction to {metadata['title']}:\n" + split.page_content
            else:
                page_content = f"{metadata['title']}\n\tsubsection:{metadata['Header_1']}:\n\tsub_subsection:{metadata['Header_2']}:\n" + split.page_content
            chunks.append(Document(page_content=page_content, metadata = metadata))
        # if i == 1:
        #     pprint(chunks)
        #     i == 2
    print(f'chunk size: {len(chunks)}')
    vectorstore = Milvus.from_documents(
        documents=chunks,
        embedding=embeddings,
        # collection_name = f'{space_key}_with_attachments',
        collection_name = space_key,
        connection_args={
            "uri": "./milvus_demo.db",},
        # Override LangChain default values for Milvus.
        consistency_level="Eventually",
        drop_old=True,
        index_params = {
            "metric_type": "COSINE",
            "index_type": "AUTOINDEX",
            "params": {}}
    )
    

    
    # for document in documents:
    #     title = document.extra_info['title']
    #     content = document.text
    #     source = 'confluence'
    #     doc_id = document.doc_id
    #     url = document.extra_info['url']
    #     # [docs.extend(Document(page_content=chunks[i], metadata = {'source':file.name, 'page': i}) for i in range(len(chunks)))]
    #     metadata = {'title': title, 
    #             'source': source, 
    #             'doc_id': doc_id, 
    #             'url': url,
    #             'Header 1': '', 
    #             'Header 2': ''
    #             }
    #     md_header_splits = markdown_splitter.split_text(content)
    #     chunks = []
    #     for i, split in enumerate(md_header_splits):
    #         metadata['sub_id'] = i
    #         metadata.update(split.metadata)
            
    #         if not (metadata['Header 1'] or metadata['Header 2']):
    #             page_content = f"Introduction to {metadata['title']}:\n" + split.page_content
    #         else:
    #             page_content = f"{metadata['title']}\n\tsubsection:{metadata['Header 1']}:\n\tsub_subsection:{metadata['Header 2']}:\n" + split.page_content
    #         chunks.extend(Document(page_content=page_content, metadata = metadata))

/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


page_content='AILAB Kimdir?  
* Yapay zeka vizyonunu ve hedefininin iletişimini yapar
* Yapay zekanın etkinleştirebileceği yeni teknolojileri, kullanım örneklerini takip ederek yenilikler yapar
* Mail grubu : AILAB Geliştirme Ekibi <ailab@softtech.com.tr>  
Miro  
https://miro.com/app/board/uXjVMowFuqc=/?share_link_id=910654917308  
Board  
AI Lab - Agile Board - Jira (softtech.com.tr)  
Discord  
https://discord.gg/5njRbhxbpP  
* * *  
### AILab ile nasıl çalışırım?  
* FrAIDay'lere katılırım ve gündemi yakın takip ederim
* Benim için tavsiye edilen eğitim ve videoları izlerim
* Duyurulardan haberdar olmak için **AI Çalışma ve Bilgilendirme <aibilgi@softtech.com.tr> **grubuna kendimi ekletirim
* AI denemelerim için kullanım yetkisi isterim
* Salı günleri 10:00-13:00 arasında sorularım için randevu isterim.  
### AI nerde kullanırım?  
* Mevcut senaryolara bakarak senaryo üretirim. .
* List contacts for getting additional help.  
### FrAIday  
Her iki haftada bir yapılan  
Join Zoom Me

In [86]:
# Load and process data separately
from pycallgraph2 import PyCallGraph
from pycallgraph2.output import GraphvizOutput

import os
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append('../..')
confluence_username = os.getenv("CONFLUENCE_USERNAME")
confluence_password = os.getenv("CONFLUENCE_PASSWORD")
from llama_index.readers.confluence import ConfluenceReader

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_milvus import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract' 
def load_data() -> dict : 


    base_url = "https://wiki.softtech.com.tr/"
    space_keys = ["YPZ"]  # ,"YNYGAB", "SPISS"]
    reader = ConfluenceReader(base_url=base_url, user_name=confluence_username, password=confluence_password)
    spaces = {}
    # def load_without_attachments():
    #     for space in space_keys:
    #         spaces[f'{space}_without_attachments'] = reader.load_data(include_attachments=False, space_key=space)
    # def load_with_attachments():
    #     for space in space_keys:
    #         spaces[f'{space}_with_attachments'] = reader.load_data(include_attachments=True, space_key=space)
    # load_without_attachments()
    # load_with_attachments()
    for space in space_keys:
        spaces[f'{space}_without_attachments'] = reader.load_data(include_attachments=False, space_key=space)
    return spaces

def process_data(spaces):

    headers_to_split_on = [
        ("#", "Header_1"),
        ("##", "Header_2")
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    for space_key, documents in spaces.items():
        print(space_key)
        chunks = []
        for document in documents:
            title = document.extra_info['title']
            content = document.text
            source = 'confluence'
            doc_id = document.doc_id
            url = document.extra_info['url']
            metadata = {
                'title': title, 
                'source': source, 
                'doc_id': doc_id, 
                'url': url,
                'Header_1': '', 
                'Header_2': ''
            }
            md_header_splits = markdown_splitter.split_text(content)
            for i, split in enumerate(md_header_splits):
                metadata['sub_id'] = i
                metadata.update(split.metadata)
                page_content = f"{metadata['title']}\n\tsubsection:{metadata['Header_1']}:\n\tsub_subsection:{metadata['Header_2']}:\n" + split.page_content
                chunks.append(Document(page_content=page_content, metadata=metadata))
        print(len(chunks))
        vectorstore = Milvus.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name=space_key,
            connection_args={"uri": "./milvus_demo.db"},
            # consistency_level="Eventually",
            drop_old=True,
            index_params={
                "metric_type": "COSINE",
                "index_type": "AUTOINDEX",
                "params": {}
            }
        )


def main():

    spaces = load_data()
    print(spaces.keys())
    process_data(spaces)


if __name__ == '__main__':

       main()


dict_keys(['YPZ_without_attachments'])
YPZ_without_attachments
90


E20240815 10:36:45.421036  5589 collection_meta.cpp:121] [SERVER][CreateCollection][grpcpp_sync_ser] Insert data failed, errs: attempt to write a readonly database
E20240815 10:36:45.421257  5589 collection_data.cpp:53] [SERVER][CreateCollection][grpcpp_sync_ser] Create table YPZ_without_attachments failed, errs: attempt to write a readonly database
RPC error: [create_collection], <MilvusException: (code=5, message=: internal error)>, <Time:{'RPC start': '2024-08-15 10:36:45.419882', 'RPC error': '2024-08-15 10:36:45.422222'}>
Failed to create collection: YPZ_without_attachments error: <MilvusException: (code=5, message=: internal error)>


MilvusException: <MilvusException: (code=5, message=: internal error)>

In [1]:
from langchain_milvus import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings

# Test documents
test_documents = ["Hello world", "This is a test document"]

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create Milvus vectorstore
vectorstore = Milvus.from_texts(
    texts=test_documents,
    embedding=embeddings,
    collection_name="test_collection",
    connection_args={"uri": "./milvus_demo.db"},
    drop_old=True,
    # index_params={
    #     "metric_type": "COSINE",
    #     "index_type": "AUTOINDEX",
    #     "params": {}
    # }
)


/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. 

# Mimari Yetkinlik Merkezi Space QA

In [83]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain_milvus import Milvus
vector_store= Milvus(
    embeddings,
    connection_args={"uri": "./milvus_demo.db"},
    collection_name='YPZ_without_attachments',
)


/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [84]:
retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
retriever.invoke("Ailab?")
# vector_store.collection.count()

[]

In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import AzureChatOpenAI

azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2024-02-15-preview"
llm_name = "gpt-4o"
system_instruction = "Based on the context provided and chat history, answer the question as an easy-to-understand chat assistant. Ensure that the answer is concise, directly addresses the question, and is in the same language as the question."
template = (
    f"{system_instruction} "
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {{chat_history}}"
    "Follow up question: {{question}}"
)
condense_question_prompt = PromptTemplate.from_template(template)

# ConversationalRetrievalChain 
qa = ConversationalRetrievalChain.from_llm(
    llm=AzureChatOpenAI(model="gpt-4o", temperature=0, api_key=azure_oai_key, azure_endpoint=azure_oai_endpoint, api_version=api_version),
    retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
)
query = "PİMS'e dahil olan modüller ne yapmalı?"
response = qa.invoke({"question": query, "chat_history": ""})

In [49]:
query = " Modüllerin PIMS servisine izinsiz erişimine izin verilmeli mi?"
response = qa.invoke({"question": query, "chat_history": ""})
print(response["answer"])

Hayır, modüllerin PIMS servisine izinsiz erişimine izin verilmemelidir. PIMS servisi, transaction bütünlüğü ve tutarlılık açısından kritik bir rol oynar. İzinsiz erişimler, bu bütünlüğü ve tutarlılığı bozabilir ve güvenlik açıklarına yol açabilir. Bu nedenle, PIMS servisine erişimlerin yetkilendirilmiş ve kontrollü bir şekilde yapılması gerekmektedir.


In [ ]:
query = " PIMS servisi nedir"
response = qa.invoke({"question": query, "chat_history": ""})
print(response["answer"])

#  Trading Plateau Ürünü Space QA

In [53]:
base_url = "https://wiki.softtech.com.tr/"
# space_keys = ["MYM"]#, "YNYGAB", "SPISS"]
space_keys = ["CAPBO"]#, "YNYGAB", "SPISS"]
reader = ConfluenceReader(base_url=base_url,  user_name=confluence_username, password=confluence_password)
spaces = {}
for space in space_keys:
    spaces[space] = reader.load_data(include_attachments=True, space_key = space)

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_milvus import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2")
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


for space_key,documents in spaces.items():
    chunks = []
    for document in documents:
        title = document.extra_info['title']
        content = document.text
        source = 'confluence'
        doc_id = document.doc_id
        url = document.extra_info['url']
        # [docs.extend(Document(page_content=chunks[i], metadata = {'source':file.name, 'page': i}) for i in range(len(chunks)))]
        metadata = {'title': title, 
                'source': source, 
                'doc_id': doc_id, 
                'url': url,
                'Header_1': '', 
                'Header_2': ''
                }
        
        md_header_splits = markdown_splitter.split_text(content)
        for i, split in enumerate(md_header_splits):
            # print("sub_id den önce", metadata)
            metadata['sub_id'] = i
            metadata.update(split.metadata)
            # print("sub_id den sonra", metadata)
            if not (metadata['Header_1'] or metadata['Header_2']):
                page_content = f"Introduction to {metadata['title']}:\n" + split.page_content
            else:
                page_content = f"{metadata['title']}\n\tsubsection:{metadata['Header_1']}:\n\tsub_subsection:{metadata['Header_2']}:\n" + split.page_content
            chunks.append(Document(page_content=page_content, metadata = metadata))
        # if i == 1:
        #     pprint(chunks)
        #     i == 2
    vectorstore = Milvus.from_documents(
        documents=chunks,
        embedding=embeddings,
        collection_name = space_key,
        connection_args={
            "uri": "./milvus_demo.db",},
        # Override LangChain default values for Milvus.
        consistency_level="Eventually",
        drop_old=False,
        index_params = {
            "metric_type": "COSINE",
            "index_type": "AUTOINDEX",
            "params": {}}
    )
    

KeyboardInterrupt: 

In [ ]:
vector_store= Milvus(
    embeddings,
    connection_args={"uri": "./milvus_demo.db"},
    collection_name="CAPBO",
)

In [ ]:
# retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
# retriever.invoke("PİMS'e dahil olan modüller ne yapmalı?")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import AzureChatOpenAI

azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2024-02-15-preview"
llm_name = "gpt-4o"
system_instruction = "Based on the context provided and chat history, answer the question as an easy-to-understand chat assistant. Ensure that the answer is concise, directly addresses the question, and is in the same language as the question."
template = (
    f"{system_instruction} "
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {{chat_history}}"
    "Follow up question: {{question}}"
)
condense_question_prompt = PromptTemplate.from_template(template)

# ConversationalRetrievalChain 
qa = ConversationalRetrievalChain.from_llm(
    llm=AzureChatOpenAI(model="gpt-4o", temperature=0, api_key=azure_oai_key, azure_endpoint=azure_oai_endpoint, api_version=api_version),
    retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
)
query = "PİMS'e dahil olan modüller ne yapmalı?"
response = qa.invoke({"question": query, "chat_history": ""})